In [13]:
import pandas as pd

# Load historical gas price data
gas_data = pd.read_csv("C:/Users/HP/Desktop/J P Morgan/Nat_Gas.csv")
gas_data['Dates'] = pd.to_datetime(gas_data['Dates'])
gas_data.set_index('Dates', inplace=True)

# Predict gas price (exact date match or nearest prior date)
def predict_gas_price(date_str):
    date = pd.to_datetime(date_str)
    if date in gas_data.index:
        return gas_data.loc[date]['Prices']
    else:
        # Find the nearest earlier date with available data
        past_dates = gas_data[gas_data.index <= date]
        if not past_dates.empty:
            return past_dates.iloc[-1]['Prices']
        else:
            raise ValueError(f"No price data available for or before {date_str}")

# Pricing model
def calculate_contract_value_model(injection_dates, withdrawal_dates,
                                   injection_rate, injection_withdrawal_costs,
                                   max_storage_volume, storage_cost_per_month):

    total_profit = 0
    avg_num_months = 30.42

    for i in range(len(injection_dates)):
        injection_price = predict_gas_price(injection_dates[i])
        print('injection_price:', injection_price)
        
        withdrawal_price = predict_gas_price(withdrawal_dates[i])
        print('withdrawal_price:', withdrawal_price)

        months_in_store = round((pd.to_datetime(withdrawal_dates[i]) - pd.to_datetime(injection_dates[i])).days / avg_num_months)
        print('months in store:', months_in_store)

        total_injected_volume = min(months_in_store * injection_rate, max_storage_volume)
        print('total_injected_volume:', total_injected_volume)

        cost_of_injection = (total_injected_volume * injection_price) - (total_injected_volume / 100000 * injection_withdrawal_costs)
        print('cost_of_injection:', cost_of_injection)

        revenue_from_sale = total_injected_volume * withdrawal_price
        print('revenue_from_sale:', revenue_from_sale)

        total_storage_cost = months_in_store * storage_cost_per_month
        print('total_storage_cost:', total_storage_cost)

        total_profit += (revenue_from_sale - cost_of_injection - total_storage_cost)

    return total_profit

# Sample parameters
injection_dates = ['10/31/2021']
withdrawal_dates = ['5/28/2025']
injection_rate = 100000  # MMBtu per month
injection_withdrawal_costs = 10000  # per MMBtu
max_storage_volume = 50000000  # MMBtu
storage_cost_per_month = 100000  # $

# Run the model
profit = calculate_contract_value_model(injection_dates, withdrawal_dates,
                                        injection_rate, injection_withdrawal_costs,
                                        max_storage_volume, storage_cost_per_month)

print("\nEstimated Contract Profit: $", round(profit, 2))


injection_price: 10.1
withdrawal_price: 11.8
months in store: 43
total_injected_volume: 4300000
cost_of_injection: 43000000.0
revenue_from_sale: 50740000.0
total_storage_cost: 4300000

Estimated Contract Profit: $ 3440000.0


C:\Users\HP\AppData\Local\Temp\ipykernel_15068\1599239051.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gas_data['Dates'] = pd.to_datetime(gas_data['Dates'])
